# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [74]:
import os
import requests
import json
import threading

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [48]:
def load_open_ai():
    load_dotenv(override=True)
    api_key = os.getenv('OPENAI_API_KEY')

    if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
        print("API key looks good so far")
    else:
        print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

    openai = OpenAI()
    MODEL_GPT = 'gpt-4o-mini'
 
    return openai, MODEL_GPT


In [49]:
def load_ollama():
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {"Content-Type": "application/json"}
    MODEL = "llama3.2"
    
    return OLLAMA_API, HEADERS, MODEL


In [50]:
system_prompt = (
    "You will be given a code snippet, which may be written in C#, Python, or Vue. "
    "Your task is to explain what the code does and why it was written that way, using clear and simple language suitable for beginner developers. "
    "Provide a step-by-step explanation and highlight any best practices or potential issues. "
    "You are an experienced and thoughtful tech lead who enjoys mentoring others. "
    "Do not introduce yourself or describe who you are in the response."
)


In [51]:
def build_prompts(code):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Please explain what this code does and why: " + code}
    ]
    return messages


In [83]:
def explain_code_open_ai(code):
    openai, model = load_open_ai()
    try:
        stream = openai.chat.completions.create(
            model=model,
            messages=build_prompts(code),
            stream=True
        )    
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)
    except Exception as e:
        print("OpenAI API Error:", e)


In [70]:
def explain_code_open_ollama(code):  
    api, headers, model = load_ollama()
    payload = {
        "model": model,
        "messages": build_prompts(code),
        "stream": False
    }

    response = requests.post(api, json=payload, headers=headers)

    if response.status_code == 200:
        data = response.json()  # convert Response to dict
        content = data["message"]["content"]  # extract content
        display(Markdown(f"### 🐑 I am **{model.upper()}**\n\n" + content))
    else:
        print(f"Error {response.status_code}: {response.text}")


In [86]:

def explain_code(code):
    # Wrap the functions with target args
    thread_ai = threading.Thread(target=explain_code_open_ai, args=(code,))
    thread_ollama = threading.Thread(target=explain_code_open_ollama, args=(code,))
    
    # Start both in parallel
    thread_ai.start()
    thread_ollama.start()
    
    # Wait for both to finish
    thread_ai.join()
    thread_ollama.join()

             
# here is the question; type over this to ask something new


In [87]:
# Type only the code you wish to get explained:
code = '{book.get("author") for book in books if book.get("author")'
explain_code(code)

API key looks good so far


This code is a Python set comprehension. Let's break it down:

1. **Understanding the Components**:
    - `books`: This is expected to be a collection (like a list) of dictionaries. Each dictionary likely represents a book and may contain multiple keys such as "title", "author", etc.
    - `book.get("author")`: This uses the `get` method for dictionaries to safely retrieve the value associated with the key "author". If the key doesn't exist, it will return `None` instead of raising a KeyError. This is a safer way to access dictionary keys, as it won't cause a program crash if "author" is missing.
    - The expression `{...}` indicates that we're creating a set. A set is a data structure that stores unique elements; so any duplicate authors will only show up once.

2. **The Comprehension Logic**:
    - `for book in books`: This loop iterates through each `book` dictionary in the `books` collection.
    - `if book.get("author")`: This condition checks whether the author exists (i.e., it's not `None` or an empty string). If there is a valid author, the author’s name will be added to the set being created.

3. **What the Code Does**:
   - This code creates a set of unique authors from the `books` collection where the "author" key exists and is not empty or `None`. 

4. **Why This is Useful**:
   - It helps in collecting unique values—in this case, the authors of books—without needing additional logic to filter out duplicates.
   - It can be particularly useful for reporting, displaying author names or further analysis where you only care about distinct authors.

#### Example:
Suppose `books` is defined as:
python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
    {"title": "Book E"}
]

When you run the given code, it will produce a set that looks like this:
python
{"Author 1", "Author 2"}

Here, "Author 1" appears twice in the list of books, but since we're using a set, it only appears once in the final output.

### 🐑 I am **LLAMA3.2**

My fellow developer!

This is a Python snippet that uses a concept called "list comprehension" or "generator expression." It's used to extract specific data from an iterable (in this case, a list of dictionaries).

Let's break it down:

1. `{... for ...}`: This is the syntax for a generator expression. It's similar to a list comprehension, but instead of creating a new list, it generates values on-the-fly.
2. `book.get("author")`: This is the expression that will be executed for each item in the iterable. In this case, we're trying to access a value called `"author"` from each dictionary (`book`) using the `get()` method.

Here's what happens when you put it all together:

* The code loops through each dictionary (`book`) in the `books` list.
* For each dictionary, it tries to retrieve the value associated with the key `"author"`.
* If the key exists, it adds that value to the generated sequence (in this case, a generator object).
* If the key doesn't exist, it skips to the next iteration.

The resulting sequence is an iterator that yields values from the original dictionaries. These values are the author names of books where the author information is available.

Here's an equivalent way to write this code using a traditional for loop:

```python
authors = []
for book in books:
    if "author" in book and book["author"] is not None:
        authors.append(book.get("author"))
```

However, the generator expression version is more concise and efficient.

**Why use this code?**

This snippet is often used when you need to process a list of dictionaries and perform some operation on each item. In this case, it's extracting author information from a list of book dictionaries.

Here are some scenarios where you might use this code:

* When working with APIs or data feeds that return JSON or XML data.
* When processing large datasets stored in databases or CSV files.
* When building data pipelines to extract and transform data.

As a best practice, it's essential to understand how list comprehensions and generator expressions work. They can make your code more concise, readable, and efficient!

Do you have any questions about this code or would you like me to explain further?

In [ ]:
# Get gpt-4o-mini to answer, with streaming

In [ ]:
# Get Llama 3.2 to answer